In [ ]:
!pip install torch==2.6.0 torchvision==0.21.0 torchaudio==2.6.0 --quiet
!pip install diffusers transformers accelerate google-generativeai pillow streamlit pyngrok langdetect --quiet
!pip install google-generativeai==0.8.5 google-ai-generativelanguage==0.6.15 --quiet

In [ ]:
!pip freeze > requirements.txt

In [ ]:
from google.colab import files
files.download("requirements.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import os

GEMINI_API_KEY = input("Enter your Google Gemini API Key: ").strip()
HF_TOKEN = input("Enter your HuggingFace Token (for Stable Diffusion): ").strip()

os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY
os.environ["HF_TOKEN"] = HF_TOKEN

In [ ]:
%%writefile app.py
import streamlit as st
from PIL import Image
import google.generativeai as genai
from diffusers import StableDiffusionPipeline
import torch
import os
from langdetect import detect

# CONFIGURE KEYS
GEMINI_API_KEY = os.environ.get("GEMINI_API_KEY")
HF_TOKEN = os.environ.get("HF_TOKEN")

# INIT MODELS
genai.configure(api_key=GEMINI_API_KEY)
text_model = genai.GenerativeModel("gemini-2.5-pro")
vision_model = genai.GenerativeModel("gemini-2.5-pro")

@st.cache_resource
def load_sd_pipeline():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    pipe = StableDiffusionPipeline.from_pretrained(
        "CompVis/stable-diffusion-v1-4",
        revision="fp16",
        torch_dtype=torch.float16 if device == "cuda" else torch.float32,
        use_auth_token=HF_TOKEN
    ).to(device)
    return pipe, device

pipe, device = load_sd_pipeline()

def chat_with_gemini_multilang(user_input):
    detected_lang = detect(user_input)
    lang_map = {
        'en': 'English',
        'es': 'Spanish',
        'fr': 'French',
        'ta': 'Tamil',
        'de': 'German',
    }
    lang = lang_map.get(detected_lang, 'English')
    # Culturally appropriate greeting (simple example)
    greetings = {
        'English': "Hello! How can I help you?",
        'Spanish': "¡Hola! ¿Cómo puedo ayudarte?",
        'French': "Bonjour! Comment puis-je vous aider?",
        'Tamil': "வணக்கம்! நான் எப்படி உதவலாம்?",
        'German': "Hallo! Wie kann ich Ihnen helfen?",
    }
    greeting = greetings.get(lang, greetings['English'])
    prompt = f"You are a helpful AI that replies in {lang} only. User says: {user_input}. Reply accordingly and culturally appropriately."
    response = text_model.generate_content(prompt)
    return f"{greeting}\n\n{response.text}"

def describe_image(img_path):
    img = Image.open(img_path)
    response = vision_model.generate_content([img, "Describe this image"])
    return response.text

def generate_image(prompt, guidance_scale=8.5):
    with torch.autocast(device):
        image = pipe(prompt, guidance_scale=guidance_scale).images[0]
    return image

# STREAMLIT UI
st.set_page_config(page_title="Multi-Language Multi-Modal Chatbot", layout="wide")
st.title("🤖 Multi-Language Multi-Modal Chatbot 🌐")

mode = st.sidebar.radio(
    "Select mode",
    ["Text Chat", "Image Understanding", "Image Generation"]
)

if mode == "Text Chat":
    st.subheader("💬 Text Chat (Multi-language) with Gemini")
    user_input = st.text_input("You:", key="text_input")
    if st.button("Send"):
        if user_input:
            response = chat_with_gemini_multilang(user_input)
            st.markdown(f"**Bot:** {response}")

elif mode == "Image Understanding":
    st.subheader("🖼️ Image Understanding with Gemini Vision")
    uploaded_file = st.file_uploader("Upload an image", type=["png", "jpg", "jpeg"])
    if uploaded_file is not None:
        img = Image.open(uploaded_file)
        st.image(img, caption="Uploaded Image", use_container_width=True)
        if st.button("Analyze Image"):
            img.save("temp_image.png")
            response = describe_image("temp_image.png")
            st.markdown(f"**Bot:** {response}")
            os.remove("temp_image.png")

elif mode == "Image Generation":
    st.subheader("🎨 Generate Image with Stable Diffusion")
    prompt = st.text_input("Enter image prompt:")
    if st.button("Generate Image"):
        if prompt:
            image = generate_image(prompt)
            st.image(image, caption="Generated Image", use_container_width=True)

st.markdown("---")
st.caption("Built with Gemini + Stable Diffusion + Streamlit")


In [ ]:
!pkill -f streamlit || echo "No Streamlit process to kill"
from pyngrok import ngrok
ngrok.kill()

NGROK_TOKEN = input("Enter your ngrok authtoken: ").strip()
os.environ["NGROK_AUTHTOKEN"] = NGROK_TOKEN

!ngrok config add-authtoken $NGROK_AUTHTOKEN

In [ ]:
from pyngrok import ngrok
import threading
import os
import time

def run_streamlit():
    os.system("streamlit run app.py > /content/logs.txt 2>&1")

thread = threading.Thread(target=run_streamlit)
thread.start()

time.sleep(10)

public_url = ngrok.connect(addr="8501", proto="http")
print(f"🌐 Streamlit app running at: {public_url}")